In [3]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinhuijiu.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [4]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,alpha,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            #print(rfe_score)
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            if len(candi_fea)==optim_featur_num:
                return candi_fea

In [5]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,0.0039)



In [42]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.778443551333577, 0.7610757346416828, 0.7901873792995459, 0.785589018215982, 0.7809906571324182, 0.7784435513335769, 0.784061015710632, 0.785592932825304, 0.7891708857456027, 0.7891617516571847, 0.7906949736416305, 0.7891604467874107, 0.7871170207213319, 0.7891565321780887, 0.7906910590323086, 0.7906897541625345, 0.7876272248029645, 0.7881387337543713, 0.7881348191450492, 0.7866015971606034, 0.7906871444229866, 0.7891565321780887, 0.7911999582441672, 0.7896680411294953, 0.7901795500809019, 0.7881374288845973, 0.7901808549506759, 0.7896693459992693, 0.7912012631139411, 0.7927370948379352, 0.7927344850983873, 0.7947779111644657, 0.7937575030012005, 0.7942664022130592, 0.7922229761469806, 0.7922242810167546, 0.7906936687718565, 0.7891630565269586, 0.7896732606085913, 0.788652852445326, 0.788652852445326, 0.7901834646902239, 0.7912077874628112, 0.7891656662665066, 0.7881400386241454, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.76

In [43]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.778443551333577, 0.7610757346416828, 0.7901873792995459, 0.7789563651547576, 0.7809906571324182, 0.7784435513335769, 0.784061015710632, 0.785592932825304, 0.7891708857456027, 0.7866159507281173, 0.7906949736416305, 0.7891604467874107, 0.788651547575552, 0.7881374288845973, 0.7881374288845973, 0.7912038728534893, 0.7876272248029645, 0.7860953076882927, 0.787115715851558, 0.7866015971606034, 0.7906871444229866, 0.788645023226682, 0.7906897541625345, 0.7911973485046192, 0.7922203664074325, 0.7906884492927605, 0.790178245211128, 0.7896693459992693, 0.7937601127407484, 0.7917114671955738, 0.7917114671955738, 0.7906910590323086, 0.7912012631139411, 0.7891591419176366, 0.7917140769351219, 0.7917179915444439, 0.7922268907563026, 0.7912064825930372, 0.7912064825930372, 0.7927370948379352, 0.7891630565269587, 0.7901834646902239, 0.7912077874628112, 0.7891656662665066, 0.7881400386241454, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.768

In [44]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.778443551333577, 0.7610757346416828, 0.7901873792995459, 0.7789563651547576, 0.7809906571324182, 0.7784435513335769, 0.784061015710632, 0.785592932825304, 0.7835508116289994, 0.7866159507281173, 0.7830406075473668, 0.7835508116289994, 0.7927423143170311, 0.78711963046088, 0.7845673051829427, 0.7835442872801295, 0.7860940028185187, 0.7901795500809019, 0.787115715851558, 0.7866055117699253, 0.7881374288845973, 0.7860953076882927, 0.7876246150634166, 0.788643718356908, 0.7876259199331906, 0.7881361240148234, 0.7927318753588392, 0.7912012631139411, 0.7881348191450492, 0.7855798841275641, 0.7850709849157054, 0.7850709849157054, 0.7845620857038468, 0.7871118012422361, 0.7906897541625346, 0.7917127720653478, 0.7922268907563026, 0.7922268907563026, 0.7906962785114046, 0.7881400386241453, 0.7891630565269587, 0.7901834646902239, 0.7891630565269586, 0.7891656662665066, 0.7881400386241454, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.768

In [45]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.778443551333577, 0.7610757346416828, 0.7901873792995459, 0.7789563651547576, 0.7809906571324182, 0.7784435513335769, 0.784061015710632, 0.785592932825304, 0.7835508116289994, 0.7866159507281173, 0.7830406075473668, 0.7835508116289994, 0.7840597108408581, 0.7906988882509525, 0.7845646954433947, 0.7860926979487448, 0.7860940028185187, 0.7901795500809019, 0.7891578370478627, 0.7901795500809018, 0.7876272248029647, 0.7871183255911058, 0.7871183255911058, 0.7876298345425127, 0.7891578370478627, 0.788646328096456, 0.7881374288845973, 0.7912012631139411, 0.7845594759642989, 0.785581188997338, 0.7850709849157054, 0.7850709849157054, 0.7845620857038468, 0.7871118012422361, 0.7906897541625346, 0.7917127720653478, 0.7917127720653478, 0.7922268907563026, 0.7906962785114046, 0.7881400386241453, 0.7876298345425127, 0.7901834646902239, 0.7891630565269586, 0.7891643613967326, 0.7881400386241454, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.7

In [46]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.778443551333577, 0.7610757346416828, 0.7901873792995459, 0.7789563651547576, 0.7809906571324182, 0.7784435513335769, 0.784061015710632, 0.7830445221566886, 0.7835508116289994, 0.7866159507281173, 0.7912103972023592, 0.7901886841693198, 0.7881478678427893, 0.7881465629730153, 0.7845673051829427, 0.7860926979487448, 0.7871157158515579, 0.7901795500809019, 0.7911986533743931, 0.7901795500809018, 0.7876272248029647, 0.7871183255911058, 0.7871183255911058, 0.7881387337543713, 0.7850762043948014, 0.7850775092645754, 0.7866042069001513, 0.788646328096456, 0.7917114671955741, 0.7871104963724621, 0.7866042069001514, 0.7840466621431182, 0.7860900882091967, 0.7860913930789707, 0.7855798841275641, 0.7855785792577901, 0.7840492718826662, 0.7876272248029645, 0.7906936687718565, 0.7881400386241453, 0.7876298345425127, 0.7901834646902239, 0.7891630565269586, 0.7891643613967326, 0.7881400386241454, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0

In [47]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.778443551333577, 0.7610757346416828, 0.7901873792995459, 0.7789563651547576, 0.7809906571324182, 0.7784435513335769, 0.784061015710632, 0.7830445221566886, 0.7835508116289994, 0.7866159507281173, 0.7912103972023592, 0.7901886841693198, 0.7881478678427893, 0.7881465629730153, 0.7845673051829427, 0.7860926979487448, 0.7871157158515579, 0.7901795500809019, 0.7911986533743931, 0.7901795500809018, 0.7876272248029647, 0.7871183255911058, 0.7860992222976148, 0.785589018215982, 0.7845646954433947, 0.7861005271673888, 0.7876337491518346, 0.7891669711362806, 0.787122240200428, 0.787122240200428, 0.7876337491518346, 0.7876337491518346, 0.7855916279555301, 0.7850827287436715, 0.7845594759642989, 0.784047967012892, 0.7840492718826662, 0.7906923639020828, 0.788645023226682, 0.7881400386241453, 0.7876298345425127, 0.7901834646902239, 0.7891630565269586, 0.7891643613967326, 0.7881400386241454, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.768

In [48]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.778443551333577, 0.7610757346416828, 0.7600605459575134, 0.7789563651547576, 0.782531708335508, 0.7784435513335769, 0.7774205334307637, 0.7789524505454356, 0.7774205334307637, 0.7763975155279501, 0.7763962106581763, 0.7794639594968422, 0.7794626546270683, 0.7764014301372724, 0.7789498408058877, 0.7876298345425127, 0.7835429824103555, 0.7881426483636933, 0.7891630565269586, 0.78405710110131, 0.7835468970196773, 0.785587713346208, 0.7850775092645753, 0.7835455921499034, 0.7820162847747795, 0.7830353880682709, 0.7850762043948014, 0.7835455921499034, 0.7815047758233727, 0.7825173547679941, 0.7855798841275641, 0.7855811889973381, 0.7891565321780887, 0.7876233101936427, 0.7860926979487448, 0.7850735946552535, 0.7840544913617621, 0.7891630565269587, 0.787118325591106, 0.7901847695599978, 0.7866055117699254, 0.7901834646902239, 0.7891630565269586, 0.7891643613967326, 0.7881400386241454, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.76

In [49]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.778443551333577, 0.7758977504045097, 0.7600605459575134, 0.7789563651547576, 0.7809906571324182, 0.7779255180333002, 0.7779307375123963, 0.7789472310663396, 0.7774205334307637, 0.7763975155279501, 0.7763962106581763, 0.775373192755363, 0.7794626546270683, 0.7764014301372724, 0.7789498408058877, 0.7876298345425127, 0.7835429824103555, 0.7820097604259095, 0.7891630565269586, 0.78405710110131, 0.7835468970196773, 0.785587713346208, 0.7850775092645753, 0.785587713346208, 0.7860966125580667, 0.7820110652956835, 0.7820136750352316, 0.7840544913617621, 0.7840544913617621, 0.7835416775405815, 0.7830301685891747, 0.7855811889973381, 0.7891565321780887, 0.7876233101936427, 0.7860926979487448, 0.7850735946552535, 0.7840544913617621, 0.7891630565269587, 0.787118325591106, 0.7901847695599978, 0.7866055117699254, 0.787115715851558, 0.788645023226682, 0.7860966125580666, 0.78558510360666, 0.7901821598204499, 0.7891617516571846, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.7749128

In [50]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.778443551333577, 0.7758977504045097, 0.7600605459575134, 0.7789563651547576, 0.7809906571324182, 0.7763988203977243, 0.7758873114463176, 0.7758833968369956, 0.775374497625137, 0.7758781773578998, 0.7779202985542044, 0.7758794822276738, 0.7774113993423456, 0.7753705830158151, 0.7769129390886789, 0.7763975155279503, 0.7789459261965656, 0.7779242131635262, 0.7815034709535988, 0.7815021660838248, 0.7820123701654575, 0.7825225742470902, 0.7825290985959601, 0.7825290985959601, 0.7794600448875203, 0.7794587400177461, 0.7794613497572942, 0.7794613497572942, 0.7789485359361136, 0.7809893522626442, 0.7809958766115142, 0.7840557962315361, 0.7820123701654575, 0.7850709849157054, 0.7809893522626442, 0.779968944099379, 0.7815008612140508, 0.7835429824103555, 0.7830314734589487, 0.7901847695599978, 0.7866055117699254, 0.787115715851558, 0.7845660003131687, 0.7860966125580667, 0.7871170207213319, 0.7866055117699253, 0.7855851036066601, 0.7891617516571847, 0.7876272248029647] [0.8037085160154135, 0.

In [61]:
np.max(aver_mean_lis1[:16]),np.max(aver_mean_lis2[:16]),np.max(aver_mean_lis3[:16]),np.max(aver_mean_lis4[:16]),np.max(aver_mean_lis5[:16]),np.max(aver_mean_lis6[:16]),np.max(aver_mean_lis7[:16]),np.max(aver_mean_lis8[:16]),np.max(aver_mean_lis9[:16])

(0.7906949736416305,
 0.7912038728534893,
 0.7927423143170311,
 0.7906988882509525,
 0.7912103972023592,
 0.7912103972023592,
 0.7876298345425127,
 0.7876298345425127,
 0.7809906571324182)

In [63]:
np.argmax(aver_mean_lis3[:16])

12

In [64]:
aver_mean_lis3[:16]

[0.778443551333577,
 0.7610757346416828,
 0.7901873792995459,
 0.7789563651547576,
 0.7809906571324182,
 0.7784435513335769,
 0.784061015710632,
 0.785592932825304,
 0.7835508116289994,
 0.7866159507281173,
 0.7830406075473668,
 0.7835508116289994,
 0.7927423143170311,
 0.78711963046088,
 0.7845673051829427,
 0.7835442872801295]

# SVM-RFE

In [6]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        kf=KFold(n_splits=5,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            if len(candi_fea)==featur_num:
                break
        return candi_fea

In [7]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.001)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [8]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.779963724620283,
  0.7534200636776449,
  0.7508625189206117,
  0.7263583694347304,
  0.7498460253666683,
  0.7493410407641318,
  0.7478104285192338,
  0.7544483010595544,
  0.7518959757816169,
  0.753430502635837,
  0.7570019312072656,
  0.7651704159924839,
  0.7682342502218278,
  0.7600644605668354,
  0.7641526175687666,
  0.7646615167806253,
  0.765174330601806,
  0.7646615167806253,
  0.7646589070410773,
  0.762103972023592,
  0.76363458426849,
  0.7600631556970615,
  0.7610796492510048,
  0.7631191607077613,
  0.7605668354298241,
  0.7610770395114568,
  0.7580119004123388,
  0.7569927971188475,
  0.7590336134453782,
  0.7590362231849261,
  0.7564812881674408,
  0.7590375280547002,
  0.7600579362179654,
  0.7595477321363328,
  0.7605681402995981,
  0.7615859387233155,
  0.7590362231849261,
  0.7595464272665587,
  0.7600566313481915,
  0.7610770395114568,
  0.76210005741427,
  0.7590336134453782,
  0.7595451223967847,
  0.7615885484628634,
  0.7615872435930895,
  0.76005532647841

In [13]:
np.argmax(rfe_SVM_aver_mean_lis)

0

# SVM-RELIEF

In [9]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(kernel='linear',C=self.C)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [10]:
relief_SVM_model=relief_SVM(x_train,y_train,0.001)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [11]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.779963724620283,
  0.765171720862258,
  0.7493384310245836,
  0.7391213006941907,
  0.7161490683229814,
  0.7156414739808967,
  0.7110431128973328,
  0.7079818884075368,
  0.7069653948535937,
  0.7095151103919829,
  0.7115559267185134,
  0.7115572315882875,
  0.716669711362806,
  0.7100292290829376,
  0.7105394331645702,
  0.7427057779633592,
  0.741686674669868,
  0.7513779424813403,
  0.7513779424813403,
  0.7493332115454878,
  0.7483114985124486,
  0.7478012944308158,
  0.7488230074638551,
  0.7539341823685997,
  0.7544456913200063,
  0.7559749986951303,
  0.7529085547262383,
  0.7564878125163108,
  0.7575095255493501,
  0.753425283156741,
  0.7554674043530455,
  0.7549545905318649,
  0.7539315726290516,
  0.7595503418758808,
  0.754953285662091,
  0.7554647946134976,
  0.7554660994832715,
  0.7549572002714129,
  0.7605733597786941,
  0.7605720549089201,
  0.7631269899264053,
  0.7600579362179654,
  0.7600592410877394,
  0.7595477321363326,
  0.7615859387233155,
  0.759034918315

In [12]:
np.argmax(relief_SVM_aver_mean_lis)

0

In [57]:
jinhui_accuracy=[]
jinhui_accuracy.append(aver_mean_lis1)
jinhui_accuracy.append(aver_mean_lis2)
jinhui_accuracy.append(aver_mean_lis3)
jinhui_accuracy.append(aver_mean_lis4)
jinhui_accuracy.append(aver_mean_lis5)
jinhui_accuracy.append(aver_mean_lis6)
jinhui_accuracy.append(aver_mean_lis7)
jinhui_accuracy.append(aver_mean_lis8)
jinhui_accuracy.append(aver_mean_lis9)
jinhui_accuracy.append(rfe_SVM_aver_mean_lis)
jinhui_accuracy.append(relief_SVM_aver_mean_lis)

In [58]:
colu=[i for i in range(1,50)]
jinhui_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(jinhui_accuracy))
jinhui_acurracy_dataframe.to_csv("jinhui_accuracy.csv")

In [59]:
jinhui_F_measure=[]
jinhui_F_measure.append(F_measure_lis1)
jinhui_F_measure.append(F_measure_lis2)
jinhui_F_measure.append(F_measure_lis3)
jinhui_F_measure.append(F_measure_lis4)
jinhui_F_measure.append(F_measure_lis5)
jinhui_F_measure.append(F_measure_lis6)
jinhui_F_measure.append(F_measure_lis7)
jinhui_F_measure.append(F_measure_lis8)
jinhui_F_measure.append(F_measure_lis9)
jinhui_F_measure.append(rfe_SVM_F_measure_lis)
jinhui_F_measure.append(relief_SVM_F_measure_lis)

In [60]:
colu=[i for i in range(1,50)]
jinhui_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(jinhui_F_measure))
jinhui_F_measure_dataframe.to_csv("jinhui_F_measure.csv")